# Policy gradient actually 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import gym
import multiprocessing
import tensorflow as tf
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv

In [3]:
'''
batch = [
    [state1, action1, reward_to_go1],
    [state2, action2, reward_to_go2] # reward_to_go2 < reward_to_go1
]

log_probs = reducesum(encoded_actions * log_softmax(logits), axis=1)
loss = -reduce_mean(rewards * log_probs)
optimizer.minimize(loss)

to desync envs: make some random steps in each 


'''

'\nbatch = [\n    [state1, action1, reward_to_go1],\n    [state2, action2, reward_to_go2] # reward_to_go2 < reward_to_go1\n]\n\nlog_probs = reducesum(encoded_actions * log_softmax(logits), axis=1)\nloss = -reduce_mean(rewards * log_probs)\noptimizer.minimize(loss)\n\nto desync envs: make some random steps in each \n\n\n'

In [12]:
ENV_NAME = 'Acrobot-v1'
NPROC = multiprocessing.cpu_count()
BATCH_SIZE = 256

In [4]:
def make_env(name, seed):
    def f():
        env = gym.make(name)
        env.seed(seed)
        return env
    return f

In [65]:
class PGAgent(object):
    
    def __init__(
        self, envs, batch_size
    ):
        self._n_envs = envs.num_envs
        self._bs = batch_size
        self._actions_size = envs.action_space.n if isinstance(envs.action_space, gym.spaces.discrete.Discrete) else envs.action_space.shape[0]
        self._observations_size = envs.observation_space.n if isinstance(envs.observation_space, gym.spaces.discrete.Discrete) else envs.observation_space.shape[0]
        self._policy = self.__create_policy()
        self._training_history = {
            'rewards': []
        }
        self._batch_gen = self.__get_batch_gen()
        
    def __get_batch_gen(self):
        while True:
            batch = np.empty((0, 3))
            trajectories = [np.empty((0, 3)) for _ in range(self._n_envs)]
            new_observations = envs.reset()
            while len(batch) <= self._bs:
                observations = new_observations
                actions = self.get_actions(observations)
                new_observations, rewards, dones, _ = envs.step(actions)
                for i in range(self._n_envs):
                    trajectories[i]  = np.concatenate((
                        trajectories[i], 
                        np.expand_dims([observations[i], actions[i], rewards[i]], axis=0)
                    ))
                    if dones[i]:
                        trajectories[i][:, 2] = self.__rew_to_go(trajectories[i][:, 2])
                        batch = np.concatenate((batch, trajectories[i][:]))
                        trajectories[i] = np.empty((0, 3))

            yield batch
                
    def __rew_to_go(self, rews, epsilon=.95):
        # cumulative discounted sum
        return np.cumsum(
            np.multiply(
                rews,
                np.array([epsilon**i for i in range(len(rews))])
            )[::-1]
        )[::-1]    
        
    def __create_policy(self):
        inp = tf.keras.layers.Input(shape=(self._observations_size,), name='input')
        x = tf.keras.layers.Dense(units=2**7, name='hidden', activation='relu')(inp)
        x = tf.keras.layers.Dense(units=self._actions_size, name='logits', activation='linear')(x)
#         x = tf.keras.layers.Activation('softmax', name='output')(x)
        model = tf.keras.models.Model([inp], [x])
        model.compile('sgd', 'mse')
        return model
    
    def get_actions(self, observations):
        preds = self._policy.predict(observations)
        actions = tf.squeeze(tf.random.categorical(preds, 1))
        return actions
    
    def train(self, n_epochs, lr=.01):
        for i in range(n_epochs):
            batch = next(self._batch_gen)
            self.__train_on_batch(batch)
            
    def __train_on_batch(self, batch):
            #  here actually do something
            print(batch[0].shape)

In [66]:
if __name__ == '__main__':
    envs = SubprocVecEnv([make_env(ENV_NAME, i) for i in range(NPROC)])    
    agent = PGAgent(
        envs, BATCH_SIZE
    )
    agent.train(1)

(3,)
